In [11]:
from tasks.purchase_invoice import  URL_BASE
from tasks.utils import login
import requests
from rich import print

In [12]:

token_access = login(
    'manager',
    '1234',
    'SBO_TDPC_PROD'
)

In [13]:
from tasks.purchase_invoice import get_business_partner

In [17]:
bp_code = ' P20550372640 '
get_business_partner('4ec310de-f5f9-11ef-8000-36426993e022', bp_code)

{'odata.metadata': 'https://192.168.208.109:50000/b1s/v1/$metadata#BusinessPartners/@Element',
 'odata.etag': 'W/"55DA3F56239D2ACF0EC75F787F59AD0673AF5057"',
 'CardCode': 'P20550372640',
 'CardName': 'RUTAS DE LIMA S.A.C',
 'CardType': 'cSupplier',
 'GroupCode': 101,
 'Address': 'AV. VICTOR ANDRES BELAUNDE NRO. 280',
 'ZipCode': None,
 'MailAddress': None,
 'MailZipCode': None,
 'Phone1': None,
 'Phone2': None,
 'Fax': None,
 'ContactPerson': None,
 'Notes': 'email ce: http://190.216.117.84/webcomprobantes/Formularios/frmInicio.aspx',
 'PayTermsGrpCode': 9,
 'CreditLimit': 0.0,
 'MaxCommitment': 0.0,
 'DiscountPercent': 0.0,
 'VatLiable': 'vLiable',
 'FederalTaxID': '20550372640',
 'DeductibleAtSource': 'tNO',
 'DeductionPercent': 0.0,
 'DeductionValidUntil': None,
 'PriceListNum': 42,
 'IntrestRatePercent': 0.0,
 'CommissionPercent': 0.0,
 'CommissionGroupCode': 0,
 'FreeText': None,
 'SalesPersonCode': -1,
 'Currency': '##',
 'RateDiffAccount': None,
 'Cellular': None,
 'AvarageLate'